In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import h5py
from pprint import pprint
import numpy as np
import logging 
logging.getLogger('matplotlib').setLevel(logging.WARNING)
import matplotlib.pyplot as plt
from scipy.special import entr
from drift_ml.datasets.bosch_cnc_machining.utils.utils import augment_xyz_samples
from drift_ml.datasets.bosch_cnc_machining.utils.dataloader import NPYBoschCNCDataLoader


In [3]:
base_loader = NPYBoschCNCDataLoader(metadata_path="/home/tbiegel/nico_files/cms_2023_code/drift_ml/src/drift_ml/datasets/bosch_cnc_machining/extracted_features/metadata_ws4096.pkl")
base_loader.load_data(
    sample_data_x_path="/home/tbiegel/nico_files/cms_2023_code/drift_ml/src/drift_ml/datasets/bosch_cnc_machining/extracted_features/npy/sample_data_x_raw_ws4096.npy",
    sample_data_y_path="/home/tbiegel/nico_files/cms_2023_code/drift_ml/src/drift_ml/datasets/bosch_cnc_machining/extracted_features/npy/sample_data_y_raw_ws4096.npy"
    )

In [4]:
sudden_config = {
    "base_config": {
        "train_size": 0.2,
        "val_size": 0.00,  # todo
        "test_size": 0.8,
        "machines": None,
        "processes": None,
        "periods": ["Feb_2019", "Aug_2019", "Feb_2020"],
    },
    "drift_config": [
        {
            "length": 25000,
            "type": "constant",
            "only_test": True,  # todo: False,
            "machines": None,
            "processes": None,
            "periods": ["Feb_2019", "Aug_2019", "Feb_2020"],
            "transform_fn": None,
        },
        {
            "length": 25000,
            "type": "constant",
            "only_test": True,  # todo: False,
            "machines": None,
            "processes": None,
            "periods": ["Aug_2020", "Feb_2021", "Aug_2021"],
            "transform_fn": None# lambda x: augment_xyz_samples(x, pitch_deg=30),
        },
    ],
}

In [5]:
from drift_ml.datasets.bosch_cnc_machining.utils.dataloader import DriftDataLoader

In [6]:
drift_dataloader = DriftDataLoader(baseloader=base_loader, config=sudden_config)

In [18]:
x, y = drift_dataloader.access_base_samples(dataset="train")

In [19]:
x.shape

(5761,)

In [42]:

chunksize = 10000
for i in range(5):
    samples, labels = drift_dataloader.access_test_drift_samples(index=i*chunksize, length=chunksize)
    
    print(samples.shape)
    print(labels.shape)

taking 10000 samples from config 0, indices 0 to 10000
skipping config 1
(10000, 4096, 3)
(10000,)
taking 10000 samples from config 0, indices 10000 to 20000
taking 0 samples from config 1, indices 0 to 0
(10000, 4096, 3)
(10000,)
skipping config 0
taking 10000 samples from config 1, indices 0 to 10000
(10000, 4096, 3)
(10000,)
skipping config 0
taking 10000 samples from config 1, indices 10000 to 20000
(10000, 4096, 3)
(10000,)
skipping config 0
skipping config 1


ValueError: need at least one array to concatenate

In [23]:
from tsfresh import feature_extraction
drift_dataloader = DriftDataLoader(baseloader=base_loader, config=sudden_config)
samples, labels = drift_dataloader.access_base_samples_tsfresh(featureset=feature_extraction.MinimalFCParameters(), dataset="test")

print(samples.shape)
print(labels.shape)

Feature Extraction:  15%|█▍        | 943/6377 [00:09<00:10, 508.74it/s]

In [8]:
incremental_config = {
    "base_config": {
        "train_size": 0.3,
        "val_size": 0.2,
        "test_size": 0.5,
        "machines": None,
        "processes": None,
        "periods": None,
    },
    "drift_config": [
        {
            "start": 0,
            "end": 10000,
            "type": "linear",
            "part_1": {
                "only_test": False,
                "machines": None,
                "processes": None,
                "periods": None,
                "transform_fn": None,
            },
            "part_2": {
                "only_test": False,
                "machines": None,
                "processes": None,
                "periods": None,
                "transform_fn": lambda x: augment_xyz_samples(x, pitch_deg=30),
            },
        }
    ],
}


In [14]:
drift_dataloader = DriftDataLoader(baseloader=base_loader, config=incremental_config)
samples = drift_dataloader.access_test_drift_samples_stft(index=0, length=10200)

print(samples.shape)

taking 10000 samples from config 0, indices 0 to 10000
(10000, 3, 129, 33)


In [38]:
import pandas as pd
from tsfresh import feature_extraction

features = pd.read_pickle(
    "/home/tbiegel/nico_files/bosch_cnc_machining/features_and_data/all_top_30_features.pkl"
)
feature_settings = feature_extraction.settings.from_columns(features)

In [19]:
drift_dataloader = DriftDataLoader(baseloader=base_loader, config=incremental_config)
samples = drift_dataloader.access_test_drift_samples_tsfresh(featureset=feature_extraction.MinimalFCParameters(), index=0, length=1000)
print(samples.shape)

taking 1000 samples from config 0, indices 0 to 1000


Feature Extraction: 100%|██████████| 300/300 [00:00<00:00, 404.89it/s]

(1000, 30)
